In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split

# Suppress Warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
customer_data = pd.read_csv('BlackFriday.csv')

In [3]:
customer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537577 entries, 0 to 537576
Data columns (total 12 columns):
User_ID                       537577 non-null int64
Product_ID                    537577 non-null object
Gender                        537577 non-null object
Age                           537577 non-null object
Occupation                    537577 non-null int64
City_Category                 537577 non-null object
Stay_In_Current_City_Years    537577 non-null object
Marital_Status                537577 non-null int64
Product_Category_1            537577 non-null int64
Product_Category_2            370591 non-null float64
Product_Category_3            164278 non-null float64
Purchase                      537577 non-null int64
dtypes: float64(2), int64(5), object(5)
memory usage: 49.2+ MB


In [4]:
customer_data = customer_data.drop(columns=['Product_Category_2', 'Product_Category_3'], inplace=False,axis=1)

In [5]:
customer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537577 entries, 0 to 537576
Data columns (total 10 columns):
User_ID                       537577 non-null int64
Product_ID                    537577 non-null object
Gender                        537577 non-null object
Age                           537577 non-null object
Occupation                    537577 non-null int64
City_Category                 537577 non-null object
Stay_In_Current_City_Years    537577 non-null object
Marital_Status                537577 non-null int64
Product_Category_1            537577 non-null int64
Purchase                      537577 non-null int64
dtypes: int64(5), object(5)
memory usage: 41.0+ MB


In [6]:
customer = customer_data.copy()

## Feature Engineering

In [16]:
customer['User_ID'] = customer['User_ID'].astype('category', copy=False)
customer['Occupation'] = customer['Occupation'].astype('category', copy=False)
customer['Marital_Status'] =  customer['Marital_Status'].map({0 : 'UnMarried', 1: 'Married'})
customer['Product_Category_1'] = customer['Product_Category_1'].astype('category', copy=False)

In [18]:
# Setting all the categorical columns to type category
for col in set(customer.columns) - set(customer.describe().columns):
    customer[col] = customer[col].astype('category')
    

print(customer.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537577 entries, 0 to 537576
Data columns (total 10 columns):
User_ID                       537577 non-null category
Product_ID                    537577 non-null category
Gender                        537577 non-null category
Age                           537577 non-null category
Occupation                    537577 non-null category
City_Category                 537577 non-null category
Stay_In_Current_City_Years    537577 non-null category
Marital_Status                537577 non-null category
Product_Category_1            537577 non-null category
Purchase                      537577 non-null int64
dtypes: category(9), int64(1)
memory usage: 10.1 MB
None


# Frequency Count for each Category

In [19]:
# feature representing the count of each user
def getCountVar(compute_df, count_df, var_name):
    grouped_df = count_df.groupby(var_name)
    count_dict = {}
    for name, group in grouped_df:
        count_dict[name] = group.shape[0]

    count_list = []
    for index, row in compute_df.iterrows():
        name = row[var_name]
        count_list.append(count_dict.get(name, 0))
    return count_list

In [20]:
customer['ID_Counts'] = getCountVar(customer, customer, 'User_ID')
customer['Product_ID_Counts'] = getCountVar(customer, customer, 'Product_ID')
customer['Age_Counts'] = getCountVar(customer, customer, 'Age')
customer['Gender_Counts'] = getCountVar(customer, customer, 'Gender')
customer['Occupation_Counts'] = getCountVar(customer, customer, 'Occupation')
customer['City_Category_Counts'] = getCountVar(customer, customer, 'City_Category')
customer['Stay_In_Current_City_Years_Counts'] = getCountVar(customer, customer, 'Stay_In_Current_City_Years')
customer['Marital_Status_Years_Counts'] = getCountVar(customer, customer, 'Marital_Status')
customer['Product_Category_1_Counts'] = getCountVar(customer, customer, 'Product_Category_1')

## Dropping User ID and Product ID

In [22]:
customer = customer.drop(columns=['User_ID', 'Product_ID'], inplace=False, axis=1)

## Polychotomization

In [24]:
features = list(customer.columns)
features.remove('Purchase')
features

['Gender',
 'Age',
 'Occupation',
 'City_Category',
 'Stay_In_Current_City_Years',
 'Marital_Status',
 'Product_Category_1',
 'ID_Counts',
 'Product_ID_Counts',
 'Age_Counts',
 'Gender_Counts',
 'Occupation_Counts',
 'City_Category_Counts',
 'Stay_In_Current_City_Years_Counts',
 'Marital_Status_Years_Counts',
 'Product_Category_1_Counts']

In [25]:
customer_polychot = pd.get_dummies(data=customer[features], drop_first=True)

## Modeling

In [30]:
x = customer_polychot.copy()
y = customer_data['Purchase']

In [32]:
## Adjusted R^2
def AdjRsquare(model, x, y):
    Rsquare = model.score(x, y)
    NoData = len(y)
    p = x.shape[1]
    tempRsquare = 1 - (1-Rsquare)*(NoData - 1
                                  )/(NoData - p - 1)
    return tempRsquare

In [34]:
## Combine all the steps to test the model performance
def linRegcheckModelPerformance(x, y):
    model = LinearRegression()
    # Covert data into train and test
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,random_state = 666, shuffle= True)
    # Build model with train data set
    model.fit(x_train, y_train)
    # Train accuracies
    trainR2 = model.score(x_train, y_train)
    predictedPurchaseTrain= model.predict(x_train)
    TrainMse = metrics.mean_squared_error(y_train, predictedPurchaseTrain)
    trainRmse = np.sqrt(TrainMse)
    trainRmsePct = trainRmse/np.mean(np.mean(np.array(y_train)))*100
    trainAdjR2 = AdjRsquare(model, x_train, y_train)
    trainAccuracies = [trainRmse, trainRmsePct, trainR2, trainAdjR2]
    # Test accuracies
    testR2 = model.score(x_test, y_test)
    predictedPurchaseTest = model.predict(x_test)
    TestMse = metrics.mean_squared_error( y_test, predictedPurchaseTest)
    testRmse = np.sqrt(TestMse)
    testRmsePct = testRmse/np.mean(np.mean(np.array(y_test)))*100
    testAdjR2 = AdjRsquare(model, x_test, y_test)
    testAccuracies = [testRmse, testRmsePct, testR2, testAdjR2]
    # Create dataframe for results
    resultsDf = pd.DataFrame(index = ["rmse", "rmsePct", "r2", "adjR2"])
    resultsDf['trainAccuracy'] = trainAccuracies
    resultsDf['testAccuracy'] = testAccuracies
    return ( round(resultsDf, 2))

In [35]:
linRegcheckModelPerformance(x,y)

,trainAccuracy,testAccuracy
rmse,2903.50,2906.65
rmsePct,31.12,31.09
r2,0.66,0.66
adjR2,0.66,0.66
